In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras #type: ignore
import tensorflow as tf #type: ignore
import random

In [2]:
DATA_PATH = r'C:\Users\kikin\OneDrive\Documents\GitHub\Cough-Sound-diagnose\dataset_full.json'
def load_data(data_path):
    with open(data_path, 'r') as f:
        data = json.load(f)
    
    X = np.array(data['mfcc'])
    y = np.array(data['labels'])
    return X, y    

In [3]:
def prepare_datasets(test_size, validation_size):
    # load data
    X, y = load_data(DATA_PATH)
    
    # create train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    # create train/validation split
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    #3d array 
    print(X_train.shape, X_validation.shape, X_test.shape)
    X_train = X_train[..., np.newaxis] 
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]
    print(X_train.shape, X_validation.shape, X_test.shape)
    
    return X_train, X_validation, X_test, y_train, y_validation, y_test

prepare_datasets(0.25, 0.2)


(262, 87, 13) (66, 87, 13) (110, 87, 13)
(262, 87, 13, 1) (66, 87, 13, 1) (110, 87, 13, 1)


(array([[[[-645.68463135],
          [ 101.17344666],
          [  10.87486839],
          ...,
          [ -20.49400711],
          [ -11.90348816],
          [  13.16423607]],
 
         [[-663.75860596],
          [  86.25136566],
          [   8.57172203],
          ...,
          [ -22.82989883],
          [ -10.51887321],
          [  21.11142731]],
 
         [[-680.73443604],
          [  65.03741455],
          [  -4.63640594],
          ...,
          [  -6.51627922],
          [  -1.68052101],
          [  26.92273712]],
 
         ...,
 
         [[-759.69543457],
          [   0.        ],
          [   0.        ],
          ...,
          [   0.        ],
          [   0.        ],
          [   0.        ]],
 
         [[-759.69543457],
          [   0.        ],
          [   0.        ],
          ...,
          [   0.        ],
          [   0.        ],
          [   0.        ]],
 
         [[-759.69543457],
          [   0.        ],
          [   0.        ],
   

In [4]:
def build_model(input_shape):
   #create model
    model = tf.keras.Sequential()
    
    #1st conv layer
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    
    #2nd conv layer
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    
    #3rd conv layer
    model.add(tf.keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    
    #flatten the output and feed it into dense layer
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    
    #output layer
    model.add(tf.keras.layers.Dense(4, activation='softmax'))
    
    return model

In [5]:
def predict(model, X, y):
    X = X[np.newaxis, ...]
    prediction = model.predict(X)
    predicted_index = np.argmax(prediction, axis=1)
    print(f'Expected index: {y}, Predicted index: {predicted_index}')

In [6]:
if __name__ == '__main__':
   #create train, validation and test sets
   X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25,0.2) #(0.25,0.2) 25% test, 20% validation

   #build the CNN net
   input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
   model = build_model(input_shape)

   #compile the network
   optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
   model.compile(optimizer=optimizer,
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

   #train the CNN
   model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)
   model.summary()

   #evaluate the CNN on the test set
   test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
   print(f'Accuracy on test set is: {test_accuracy}')

   #make predictions on a sample
   

(262, 87, 13) (66, 87, 13) (110, 87, 13)
(262, 87, 13, 1) (66, 87, 13, 1) (110, 87, 13, 1)
Epoch 1/30


c:\Users\kikin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.1864 - loss: 1.9074 - val_accuracy: 0.2879 - val_loss: 2.9853
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2811 - loss: 1.6961 - val_accuracy: 0.2879 - val_loss: 2.3522
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2792 - loss: 1.6058 - val_accuracy: 0.2879 - val_loss: 2.0748
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3414 - loss: 1.4808 - val_accuracy: 0.2879 - val_loss: 1.8728
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3291 - loss: 1.3936 - val_accuracy: 0.2879 - val_loss: 1.7286
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4125 - loss: 1.2480 - val_accuracy: 0.2879 - val_loss: 1.6234
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4344 - loss: 1.2096 - val_accuracy: 0.2879 - val_loss: 1.5593
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5099 - loss: 1.0861 - val_accuracy: 0.3030 - val_loss: 1.5092
Epoch 9/30


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 85, 11, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 43, 6, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 43, 6, 32)      │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 41, 4, 32)      │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 21, 2, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 21, 2, 32)      │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 1, 32)      │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 10, 1, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 10, 1, 32)      │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 320)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 104,270 (407.31 KB)

 Trainable params: 34,692 (135.52 KB)

 Non-trainable params: 192 (768.00 B)

 Optimizer params: 69,386 (271.04 KB)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5203 - loss: 0.9667 
Accuracy on test set is: 0.5272727012634277


In [7]:
predict(model, X_test[random.randint(0, len(X_test) - 1)], y_test[random.randint(0, len(y_test) - 1)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Expected index: 1, Predicted index: [3]


In [8]:
model.save('cough_classifier.h5')